In [ ]:
import psycopg2
import undetected_chromedriver as uc
uc.install()
from selectorlib import Extractor
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import WebDriverException
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver import Chrome
from fake_useragent import UserAgent
import time
import random

import pandas as pd
import numpy as np

# Obtain information from local database.

In [ ]:
#make connection with local DB
conn = psycopg2.connect(host="localhost", port = 5432, database="addresses", user="postgres", password="password")

# Create a cursor object
cur = conn.cursor()

# A sample query of all data from the "vendors" table in the "suppliers" database
cur.execute("""SELECT * FROM house WHERE post_comm ilike('hackettstown')""")
query_results = cur.fetchall()
#print(query_results)

# Close the cursor and connection to so the server can allocate
# bandwidth to other requests
cur.close()
conn.close()

In [ ]:
random.uniform(.1,.3)

# Initiatial structure

An initial proof of concept script to play around with different strategies for data collection. This basic loop grabs all the information using a rotating proxy. 

In [ ]:
for i, house in enumerate(query_results):
    position = i+=1
    
    capa = DesiredCapabilities.CHROME
    capa["pageLoadStrategy"] = 'none'
    
    #### Start browser
    #ChromeOptions = webdriver.ChromeOptions()
    ChromeOptions = uc.ChromeOptions()
    ua = UserAgent()
    userAgent = ua.random
    print(userAgent)
    ChromeOptions.add_argument('--incognito')
    ChromeOptions.add_argument(f'user-agent={userAgent}')
    ChromeOptions.add_argument('--headless')
    ChromeOptions.add_argument('--proxy-server=198.204.241.50:17057')
    
    
    #executable_path=ChromeDriverManager().install()
    #driver = webdriver.Chrome(options=ChromeOptions, desired_capabilities=capa)
    driver = Chrome(options=ChromeOptions, desired_capabilities=capa)
    driver.delete_all_cookies()

    
    time.sleep(random.uniform(.1,3.75))
    

    
    
    #establish database connection
    conn = psycopg2.connect(host="localhost", port = 5432, database="addresses", user="postgres", password="password")
    town = house[1]
    street = house[8]
    zipcode = house[7]
    num =house[4]
    st = house[2]
    state = house[0]
    address_to_input = street + ' ' + ' ' + town + ' ' + state + ' ' + zipcode
    address_to_input = address_to_input.replace('  ', ' ').lower()
    address_to_input = address_to_input.replace(' ', '-').lower()
    time.sleep(.6)
    #direct link
    
    timeout = random.uniform(62,88)
    wait = WebDriverWait(driver, timeout)
    
    
    try:
        driver.get(f"https://thatsthem.com/address/{address_to_input}")
        wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="app"]/div/main/section[1]/div/form/div/button')))

        time.sleep(random.uniform(.1,2.3))
        driver.execute_script("window.stop();")
        print('element loaded')
    except TimeoutException:
        print("Timed out waiting for page to load!")
    except WebDriverException as e:
        print('SOMETHING WENT WRONG', e)
        pass
    except Exception as e:
        print('Error: !!!!!: ', e)
        pass
    finally:
        print("Address: ", street)
    
    time.sleep(random.uniform(.1,.3))
    raw_html = driver.page_source
    
    resident_info = Extractor.from_yaml_string("""
    main_card:
        css: 'div.ThatsThem-record:nth-of-type(n+2) div.ThatsThem-well'
        xpath: null
        multiple: true
        type: Text
        children:
            address:
                css: div.ThatsThem-record-address
                xpath: null
                multiple: true 
                type: Text
            phone_number:
                css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(1)'
                xpath: null
                multiple: true
                type: Text
            alt_phone:
                css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(2)'
                xpath: null
                multiple: true
                type: Text
            email:
                css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(3)'
                xpath: null
                multiple: true
                type: Text
            length_of_residence:
                css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(4)'
                xpath: null
                type: Text
            household_size:
                css: 'div.col-md-6:nth-of-type(1) dd.col-md-8:nth-of-type(5)'
                xpath: null
                multiple: true
                type: Text
            ip_address:
                css: 'dd.col-md-8:nth-of-type(6)'
                xpath: null
                multiple: true
                type: Text
            net_worth:
                css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(1)'
                xpath: null
                multiple: true
                type: Text
            estimated_income:
                css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(2)'
                xpath: null
                multiple: true
                type: Text
            education:
                css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(3)'
                xpath: null
                multiple: true
                type: Text
            occupation:
                css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(4)'
                xpath: null
                multiple: true
                type: Text
            language:
                css: 'div.col-md-6:nth-of-type(2) dd.col-md-8:nth-of-type(5)'
                xpath: null
                multiple: true
                type: Text
            name:
                css: 'h2 span'
                xpath: null
                multiple: true
                type: Text
            """)
    raw_data = resident_info.extract(raw_html)
    
    df = pd.DataFrame(raw_data['main_card'])
    
    df = df.apply(unlist)
    
    df['address_id'] = house[-1]
    
    for row in list(range(len(df))):
        data = df.iloc[row].values

        address_id = str(data[-1])
        resident_phone = str(data[1])
        alt_phones = str(data[2])
        email = str(data[3])
        resident_length = str(data[4])
        household_size = str(data[5])
        ip_address = str(data[6])
        net_worth = str(data[7])
        estimated_income = str(data[8])
        education = str(data[9])
        occupation = str(data[10])
        name = str(data[12])
        print("saving: ", name)

        cur = conn.cursor()

        cur.execute("""INSERT INTO residents(address_id, resident_phone, alt_phones, email, resident_length, household_size, ip_address, net_worth, estimated_income, education, occupation, name) VALUES
        (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);""",(address_id, resident_phone, alt_phones, email, resident_length, household_size, ip_address, net_worth, estimated_income, education, occupation, name))
        conn.commit()

        cur.close()
    conn.close()
    time.sleep(random.uniform(.1,.3))
    driver.close()

In [ ]:
def unlist(entry):
    entry = entry.str[0]
    return entry
    

# Tor

Playing around with using Tor as a proxy for selenium. 

In [ ]:
from stem import Signal
from stem.control import Controller
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from bs4 import BeautifulSoup

# signal TOR for a new connection
def switchIP():
    with Controller.from_port(port = 9051) as controller:
        controller.authenticate()
        controller.signal(Signal.NEWNYM)

# get a new selenium webdriver with tor as the proxy
def my_proxy(PROXY_HOST,PROXY_PORT):
    fp = webdriver.FirefoxProfile()
    # Direct = 0, Manual = 1, PAC = 2, AUTODETECT = 4, SYSTEM = 5
    fp.set_preference("network.proxy.type", 1)
    fp.set_preference("network.proxy.socks",PROXY_HOST)
    fp.set_preference("network.proxy.socks_port",int(PROXY_PORT))
    fp.update_preferences()
    options = Options()
    #options.headless = True
    return webdriver.Firefox(options=options, firefox_profile=fp)

for i, house in enumerate(query_results[1146:]):
    position = i+1146
    
    
    proxy = my_proxy("127.0.0.1", 9050)
#     proxy.get("https://whatsmyip.com/")
#     html = proxy.page_source
    

    #proxy.get(f"http://www.addresses.com/addr/{address_to_input}")
    proxy.get('https://www.britannica.com/')
    
    raw_html = proxy.page_source
    
    soup = BeautifulSoup(raw_html, 'lxml')
    #print(soup.find("h4", {"class": "title"}).text)
    proxy.close()
    switchIP()